# Armado de Base de Datos

## Cargo paquetes necesarios

In [9]:
from datasets import load_dataset, Dataset
from huggingface_hub import HfApi, HfFolder, notebook_login
from nltk.tokenize import sent_tokenize

import pandas as pd
import spacy

nlp = spacy.load('es_core_news_sm')

## Cargo base de datos de huggingface

In [10]:
d1 = load_dataset('Danielbrdz/Barcenas-HumorNegro', split='train')
print(d1)
d2 = load_dataset('xaviviro/chistes_eugenio', split='train')
print(d2)
d3 = load_dataset('mrm8488/CHISTES_spanish_jokes', split='train')
print(d3)
d4 = pd.read_csv('jokes.txt', sep='\t', encoding='utf-8')
d4.columns = ['joke']
# lo convierto en un dataset de huggingface
d4 = Dataset.from_pandas(d4)
print(d4)

Dataset({
    features: ['Chiste', 'Explicación'],
    num_rows: 500
})
Dataset({
    features: ['result', 'chiste'],
    num_rows: 101
})
Dataset({
    features: ['id', 'text', 'keywords', 'funny', 'category'],
    num_rows: 2419
})
Dataset({
    features: ['joke'],
    num_rows: 975
})


## Preproceso de datos

Solo me voy a quedar con los chistes que tienen la siguiente estructura:

[QUESTION] [ANSWER]

In [11]:
def extract_qa_jokes_spacy(dataset, joke_column):
    '''
    Extrae los indices de chistes que cumplen la condicion de tener 
    entre 2 y 3 oraciones y que comiencen con una pregunta.

    Parámetros:
    - dataset: El dataset que contiene los chistes.
    - joke_column: Nombre de la columna donde están los chistes.

    Retorna:
    - Lista de indices de los chistes que cumplen con las condiciones.
    '''
    qa_jokes_indices = []

    # Iterar sobre las entradas del dataset
    for i, joke in enumerate(dataset[joke_column]):
        # Tokenizacion usando spaCy
        doc = nlp(joke.strip())
        sentences = [sent.text for sent in doc.sents]

        # Verificar condiciones: entre 2 y 3 oraciones, y la primera termina con '?'
        if 1 < len(sentences) < 4 and sentences[0].strip().endswith('?'):
            qa_jokes_indices.append(i)

    return qa_jokes_indices

In [12]:
qa_jokes_d1 = extract_qa_jokes_spacy(d1, 'Chiste')
qa_jokes_d2 = extract_qa_jokes_spacy(d2, 'chiste')
qa_jokes_d3 = extract_qa_jokes_spacy(d3, 'text')
qa_jokes_d4 = extract_qa_jokes_spacy(d4, 'joke')

In [13]:
# Imprimir la cantidad de chistes que cumplen con las condiciones
print(f'Cantidad de chistes en d1: {len(qa_jokes_d1)}')
print(f'Cantidad de chistes en d2: {len(qa_jokes_d2)}')
print(f'Cantidad de chistes en d3: {len(qa_jokes_d3)}')
print(f'Cantidad de chistes en d4: {len(qa_jokes_d4)}')

# imprimo un chiste ramdom de cada dataset
print('')
print(d1['Chiste'][qa_jokes_d1[0]])
print('')
print(d2['chiste'][qa_jokes_d2[0]])
print('')
print(d3['text'][qa_jokes_d3[0]])
print('')
print(d4['joke'][qa_jokes_d4[0]])

Cantidad de chistes en d1: 482
Cantidad de chistes en d2: 11
Cantidad de chistes en d3: 338
Cantidad de chistes en d4: 900

¿Cuál es el beneficio de salir con una visca? - Que mientras te chupa laicha, te cuida la moto. Jaja

Dos amigos y uno le dice al otro:
– Oye, ¿y tu mujer cuando hace el amor grita?
– ¿Que si grita? ¡La oigo desde el bar!


- ¿Cuál es el mejor portero del mundial? 
- Evidente ¡el de Para-guay!

Puedo afeitar todo el día, pero mi barba sigue viéndose igual ¿Qué soy? un barbero


In [14]:
def extract_qa_pairs(dataset, joke_column, qa_indices):
    '''
    Extrae preguntas y respuestas de los chistes en el dataset segun los indices proporcionados.

    Parametros:
    - dataset: El dataset que contiene los chistes.
    - joke_column: Nombre de la columna donde están los chistes.
    - qa_indices: Lista de índices de los chistes QA seleccionados.

    Retorna:
    - Un diccionario con las preguntas y respuestas extraídas.
    '''
    jokes = {'Question': [], 'Answer': []}

    for idx in qa_indices:
        # Accede directamente a la fila y columna
        joke = dataset[idx][joke_column]
        doc = nlp(joke.strip())
        sentences = [sent.text for sent in doc.sents]  # Tokenización con spaCy

        if len(sentences) >= 2:
            question = sentences[0]
            answer = ' '.join(sentences[1:])
            jokes['Question'].append(question)
            jokes['Answer'].append(answer)

    return jokes

In [15]:
# Supongamos que 'text' es la columna con los chistes en d1
jokes_d1 = extract_qa_pairs(d1, 'Chiste', qa_jokes_d1)
jokes_d2 = extract_qa_pairs(d2, 'chiste', qa_jokes_d2)
jokes_d3 = extract_qa_pairs(d3, 'text', qa_jokes_d3)
jokes_d4 = extract_qa_pairs(d4, 'joke', qa_jokes_d4)

In [16]:
START_DOC_TOKEN = ''
END_DOC_TOKEN = '<|endoftext|>'


def create_qa_corpus(dataset, qa_indices, joke_column):
    '''
    Crea un corpus de preguntas y respuestas a partir de los chistes.

    Parametros:
    - dataset: Dataset que contiene los chistes.
    - qa_indices: Lista de índices de chistes para procesar.
    - joke_column: Nombre de la columna que contiene los chistes.

    Retorna:
    - Un string que representa el corpus de preguntas y respuestas.
    '''
    qa_corpus = ""
    START_DOC_TOKEN = "<START>"
    END_DOC_TOKEN = "<END>"

    for idx in qa_indices:
        # Accede al chiste usando el indice
        joke = dataset[idx][joke_column]  # Usa el nombre de la columna
        doc = nlp(joke.strip())

        # Tokeniza las oraciones
        sentences = [sent.text for sent in doc.sents]

        if len(sentences) >= 2:
            question, answer = sentences[0], ' '.join(sentences[1:])
            qa_corpus += f'{START_DOC_TOKEN}[QUESTION] {question}\n[ANSWER] {answer}\n{END_DOC_TOKEN}\n'

    return qa_corpus

In [17]:
# Crear el corpus de preguntas y respuestas
qa_corpus_d1 = create_qa_corpus(d1, qa_jokes_d1, 'Chiste')
qa_corpus_d2 = create_qa_corpus(d2, qa_jokes_d2, 'chiste')
qa_corpus_d3 = create_qa_corpus(d3, qa_jokes_d3, 'text')
qa_corpus_d4 = create_qa_corpus(d4, qa_jokes_d4, 'joke')

In [18]:
# junto los 3 corpus en uno solo
qa_corpus = qa_corpus_d1 + qa_corpus_d2 + qa_corpus_d3 + qa_corpus_d4
qa_corpus

'<START>[QUESTION] ¿Cuál es el beneficio de salir con una visca?\n[ANSWER] - Que mientras te chupa laicha, te cuida la moto. Jaja\n<END>\n<START>[QUESTION] ¿Cuál es la diferencia entre Jesucristo y un retrato de Jesucristo?\n[ANSWER] Pues el retrato necesita un clavo.\n<END>\n<START>[QUESTION] ¿Cuál es la diferencia entre una mujer y dos hombres sin brazos?\n[ANSWER] Que ninguno tiene derechos.\n<END>\n<START>[QUESTION] Hay un nazi que esta por ejecutar a un negro y un judío, ¿a quien mata primero y por que?\n[ANSWER] - Mata primero al judío, porque el trabajo es primero y la diversión después\n<END>\n<START>[QUESTION] Qué es lo más difícil de ocultar en una piscina de bebés muertos?\n[ANSWER] - La erección\n<END>\n<START>[QUESTION] Porque el golf es el deporte favorita de los curas?\n[ANSWER] Porque la mayoría de los hoyos son menores de 18\n<END>\n<START>[QUESTION] Que diferencia hay entre un cura y el acné?\n[ANSWER] Que el acné espera a que tengas 14 años\n<END>\n<START>[QUESTION] 

In [19]:
def split_corpus_to_dataframe(corpus):
    '''
    Funcion que separa un texto usando '<END>\n' como separador y lo convierte en un DataFrame.
    Mantiene el '<END>' al final de cada chiste.

    Args:
        corpus (str): El texto a separar.

    Returns:
        pd.DataFrame: DataFrame con una columna llamada 'joke'.
    '''
    # Separar el corpus en función del separador '<END>\n' y mantener el separador
    jokes = [joke + '<END>' for joke in corpus.split('<END>\n')]

    # Crear un DataFrame con los chistes
    df = pd.DataFrame(jokes, columns=['joke'])

    return df


# Ejemplo de uso
df = split_corpus_to_dataframe(qa_corpus)
print(df)

                                                   joke
0     <START>[QUESTION] ¿Cuál es el beneficio de sal...
1     <START>[QUESTION] ¿Cuál es la diferencia entre...
2     <START>[QUESTION] ¿Cuál es la diferencia entre...
3     <START>[QUESTION] Hay un nazi que esta por eje...
4     <START>[QUESTION] Qué es lo más difícil de ocu...
...                                                 ...
1727  <START>[QUESTION] ¿Sabes qué le dice una piedr...
1728  <START>[QUESTION] ¿Por qué está tan feliz siem...
1729  <START>[QUESTION] ¿Qué le dice un gusano a otr...
1730  <START>[QUESTION] ¿Cuál es el colmo de una cab...
1731                                              <END>

[1732 rows x 1 columns]


In [20]:
# Guardar el corpus en un archivo csv
df.to_csv('jokes_tm.csv', index=False)

In [21]:
# inicio sesion en huggingface
notebook_login()

In [22]:
# Define el nombre del dataset y tu usuario
dataset_name = 'kevmansilla/jokes_spanish_tm'
file_path = 'jokes_tm.csv'

# Crea el API
api = HfApi()

# Carga el archivo al repositorio
api.upload_file(
    path_or_fileobj=file_path,
    path_in_repo='jokes_tm.csv',
    repo_id=dataset_name,
    repo_type='dataset',
)

print('Archivo subido con éxito.')

Archivo subido con éxito.
